# Import Library

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [ ]:
tf.debugging.set_log_device_placement(False)
tf.config.set_soft_device_placement(False)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2 Physical GPUs, 2 Logical GPUs


In [ ]:
tf.test.is_built_with_cuda()

True

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [ ]:
import os
#import cv2
import sys
import gc
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling2D, AveragePooling2D, Flatten, Dense, LSTM, TimeDistributed, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import keras.utils as image

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
gc.collect()

0

# Proprocessing

## Set Constant

In [ ]:
# Define constants
#DATA_DIR = 'Validate'
DATA2_DIR = '/home/ycaun/Desktop/jeheng/Train2'
CATEGORIES = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv','yoga']
NUM_CLASSES = len(CATEGORIES)
INPUT_SHAPE = (40, 112, 112, 3)  # (frames, height, width, channels)
BATCH_SIZE = 5
EPOCHS = 20

## Define function

In [ ]:
def preprocess_data(category, GPU):
    tf.debugging.set_log_device_placement(True)
    tf.config.set_soft_device_placement(False)
    with tf.device(GPU):

        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a,b)


        X = []
        y = []
        exampleCount = 0
        vidcount = 0
        prev_step = 0
        category_dir = os.path.join(DATA2_DIR, category)
        print("Loading ", category)

        for video in os.listdir(category_dir):
            video_dir = os.path.join(category_dir, video)


            if vidcount % 100 != 0 or vidcount == 0:
                if len(os.listdir(video_dir)) > 0:
                    print("Loading video ",str(exampleCount), ': ', str(video))
                    #print(str(vidcount % 500))
                    frames = []

                    for frame_file in range(40):
                        img_path = os.path.join(video_dir, str(str(frame_file) + '.jpg'))
                        img = image.load_img(img_path, target_size=(224, 224))
                        x = image.img_to_array(img)
                        x = x/255
                        x = np.expand_dims(x, axis=0)
                        frames.append(x)
                    X.append(np.concatenate(frames))
                    y.append(category)
                    exampleCount += 1
                    frames.clear()
                    gc.collect()

                    for frame_file in range(40,80):
                        img_path = os.path.join(video_dir, str(str(frame_file) + '.jpg'))
                        img = image.load_img(img_path, target_size=(224, 224))
                        x = image.img_to_array(img)
                        x = x/255
                        x = np.expand_dims(x, axis=0)
                        frames.append(x)
                    X.append(np.concatenate(frames))
                    y.append(category)
                    exampleCount += 1
                    frames.clear()
                    gc.collect()

                    # for frame_file in range(80,120):
                    #     img_path = os.path.join(video_dir, str(str(frame_file) + '.jpg'))
                    #     img = image.load_img(img_path, target_size=(224, 224))
                    #     x = image.img_to_array(img)
                    #     x = x/255
                    #     x = np.expand_dims(x, axis=0)
                    #     frames.append(x)
                    # X.append(np.concatenate(frames))
                    # y.append(category)
                    # exampleCount += 1
                    # frames.clear()
                    # gc.collect()

                    vidcount += 1

                else:
                    print('Skip vid: ', video)
            else:
                print('\nLoaded batch' + str(exampleCount))
                print("Formating input X")
                X_np = np.array(X)
                print("Formating label y")
                y_np = np.array(y)
                print("Shape of X: ", X_np.shape)
                print("Shape of y: ", y_np.shape)
                np.save('/home/ycaun/Desktop/jeheng/Numpy/' + category + '_batch' + str(exampleCount), X_np)
                np.save('/home/ycaun/Desktop/jeheng/Numpy/' + category + '_label_batch' + str(exampleCount), y_np)
                X.clear()
                y.clear()
                del X_np, y_np
                gc.collect()
                vidcount = 0

        print('\nSummary: \n')
        print(str(category), ': ', str(exampleCount))

    return 0

In [ ]:
def preprocess_data(category, GPU):
    tf.debugging.set_log_device_placement(True)
    tf.config.set_soft_device_placement(False)
    with tf.device(GPU):
        X = []
        y = []
        exampleCount = 1
        category_dir = os.path.join(DATA2_DIR, category)
        print("Loading ", category)

        for video in os.listdir(category_dir):
            video_dir = os.path.join(category_dir, video)

            if len(os.listdir(video_dir)) > 0:
                print("Loading video ",str(exampleCount), ': ', str(video))
                frames = []

                for frame_file in range(40):
                    img_path = os.path.join(video_dir, str(str(frame_file) + '.jpg'))
                    img = image.load_img(img_path, target_size=(112, 112))
                    x = image.img_to_array(img)
                    x = x/255
                    x = np.expand_dims(x, axis=0)
                    frames.append(x)
                X.append(np.concatenate(frames))
                y.append(category)
                exampleCount += 1
                frames.clear()
                gc.collect()

                for frame_file in range(40,80):
                    img_path = os.path.join(video_dir, str(str(frame_file) + '.jpg'))
                    img = image.load_img(img_path, target_size=(112, 112))
                    x = image.img_to_array(img)
                    x = x/255
                    x = np.expand_dims(x, axis=0)
                    frames.append(x)
                X.append(np.concatenate(frames))
                y.append(category)
                exampleCount += 1
                frames.clear()
                gc.collect()

            else:
                print('Skip vid: ', video)

        print("Finished loading ", category)
        print("Formating input X")
        X = np.array(X)
        print("Formating label y")
        y = np.array(y)

        print('\nSummary: \n')
        print(str(category), ': ', str(exampleCount))

    return X, y

In [ ]:
def extract_feature(X, model, step):
    tf.debugging.set_log_device_placement(False)
    tf.config.set_soft_device_placement(True)
    with tf.device('/device:GPU:1'):

        start = 0
        end = 0
        while end < X.shape[0]:
            end = start + step
            if start != 0:
                print('Processing video ', start, ' - ', end)
                X_sliced = X[start:end,:,:,:,:]
                X_train = model.predict_on_batch(X_sliced)
                X_train_concat = np.concatenate((X_train_concat,X_train))
                start = end
                gc.collect()
            else:
                print('Start processing...')
                print('Processing video ', start, ' - ', end)
                X_sliced = X[start:end,:,:,:,:]
                X_train = model.predict_on_batch(X_sliced)
                X_train_concat = X_train
                start = end
                gc.collect()

        print('Processed ', end, ' video')

    return X_train_concat

In [ ]:
gc.collect()

0

In [ ]:
def train_test_load_data(categories, X_train, X_test):
    if len(X_train) != 0 and len(X_test) != 0:
        x_processed = np.load('f:/Kinetic_700/numpy2/' + categories + '_processed.npy')
        split = round(len(x_processed)*0.8)
        X_train = np.concatenate((X, x_processed[0:split]))
        X_test = np.concatenate((X, x_processed[split:len(x_processed)]))
        return X_train, X_test, x_processed.shape, split
    else:
        x_processed = np.load('f:/Kinetic_700/numpy2/' + categories + '_processed.npy')
        split = round(len(x_processed)*0.8)
        X_train = x_processed[0:split]
        X_test = x_processed[split:len(x_processed)]
        return X_train, X_test, x_processed.shape, split

In [ ]:
def train_test_load_label(categories, y_train, y_test):
    if len(y_train) != 0 and len(y_test) != 0:

        y_processed = np.load('f:/Kinetic_700/numpy2/' + categories + '_label.npy')
        split = round(len(y_processed)*0.8)

        y_train = np.concatenate((y_train, y_processed[0:split]))
        y_test = np.concatenate((y_train, y_processed[split:len(y_processed)]))

        return y_train, y_test, y_processed.shape, split
    else:
        y_processed = np.load('f:/Kinetic_700/numpy2/' + categories + '_label.npy')
        split = round(len(y_processed)*0.8)

        y_train = y_processed[0:split]
        y_test = y_processed[split:len(y_processed)]
        return y, y.shape

## Define extractor model

In [ ]:
# creating the base model of pre-trained VGG16 model
from keras.applications.vgg16 import VGG16

### ============================ VGG16 GlobalAVG  ============================

In [ ]:
tf.debugging.set_log_device_placement(False)
tf.config.set_soft_device_placement(False)
with tf.device('/device:GPU:1'):
    # Load vgg16 model
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

    # Freeze vgg16 layers
    for layer in vgg16.layers:
        layer.trainable = False

    # Extract features using vgg16
    inputs = Input(shape=INPUT_SHAPE)
    l1 = TimeDistributed(vgg16)(inputs)
    l2 = TimeDistributed(GlobalAveragePooling2D())(l1)
    #outputs = TimeDistributed(Flatten())(l2)

    # Create model
    vgg16_gavg = Model(inputs, l2)
    vgg16_gavg.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 40, 112, 112, 3)  0         
                             ]                                   
                                                                 
 time_distributed_3 (TimeDis  (None, 40, 3, 3, 512)    14714688  
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 40, 512)          0         
 tributed)                                                       
                                                                 
Total params: 14,714,688
Trainable params: 0
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
gc.collect()

2253

### ============================ VGG16 AVG ============================

In [ ]:
# Load vgg16 model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

# Freeze vgg16 layers
for layer in vgg16.layers:
    layer.trainable = False

# Extract features using vgg16
inputs = Input(shape=INPUT_SHAPE)
l1 = TimeDistributed(vgg16)(inputs)
l2 = TimeDistributed(AveragePooling2D(pool_size=(2,2),strides=1))(l1)
outputs = TimeDistributed(Flatten())(l2)

# Create model
vgg16_avg = Model(inputs, outputs)
vgg16_avg.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 40, 224, 224, 3)  0         
                             ]                                   
                                                                 
 time_distributed_2 (TimeDis  (None, 40, 7, 7, 512)    14714688  
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 40, 6, 6, 512)    0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 40, 18432)        0         
 tributed)                                                       
                                                                 
Total params: 14,714,688
Trainable params: 0
Non-trainable 

### ============================ Resnet GlobalAVG ============================

In [ ]:
# Load ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

# Freeze ResNet50 layers
for layer in resnet.layers:
    layer.trainable = False

# Extract features using ResNet50
inputs = Input(shape=INPUT_SHAPE)
l1 = TimeDistributed(resnet)(inputs)
l2 = TimeDistributed(GlobalAveragePooling2D())(l1)
#outputs = TimeDistributed(Flatten())(l2)

# Create model
resnet_gavg = Model(inputs, l2)
resnet_gavg.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 40, 112, 112, 3)  0         
                             ]                                   
                                                                 
 time_distributed_6 (TimeDis  (None, 40, 4, 4, 2048)   23587712  
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 40, 2048)         0         
 tributed)                                                       
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


### ============================ EfficientnetB7 GlobalAVG ============================

In [ ]:
# creating the base model of pre-trained EffB7 model
from keras.applications import EfficientNetB7

In [ ]:
tf.debugging.set_log_device_placement(False)

In [ ]:
tf.debugging.set_log_device_placement(False)
tf.config.set_soft_device_placement(False)
with tf.device('/device:GPU:1'):
    # Load ResNet50 model
    effB7 = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

    # Freeze ResNet50 layers
    for layer in effB7.layers:
        layer.trainable = False

    # Extract features using ResNet50
    inputs = Input(shape=INPUT_SHAPE)
    l1 = TimeDistributed(effB7)(inputs)
    l2 = TimeDistributed(GlobalMaxPooling2D())(l1)
    #outputs = TimeDistributed(Flatten())(l2)

    # Create model
    effB7_gavg = Model(inputs, l2)
    effB7_gavg.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 40, 112, 112, 3)  0         
                             ]                                   
                                                                 
 time_distributed (TimeDistr  (None, 40, 4, 4, 2560)   64097687  
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 40, 2560)         0         
 tributed)                                                       
                                                                 
Total params: 64,097,687
Trainable params: 0
Non-trainable params: 64,097,687
_________________________________________________________________


### ============================ InceptionResNetV2 GlobalAVG ============================

In [ ]:
# creating the base model of pre-trained EffB7 model
from keras.applications import InceptionResNetV2

In [ ]:
tf.debugging.set_log_device_placement(False)
tf.config.set_soft_device_placement(False)
with tf.device('/device:GPU:1'):
    # Load InceptionResNetV2 model
    irnV2 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

    # Freeze ResNet50 layers
    for layer in irnV2.layers:
        layer.trainable = False

    # Extract features using ResNet50
    inputs = Input(shape=INPUT_SHAPE)
    l1 = TimeDistributed(irnV2)(inputs)
    l2 = TimeDistributed(GlobalAveragePooling2D())(l1)
    #outputs = TimeDistributed(Flatten())(l2)

    # Create model
    irnV2_gavg = Model(inputs, l2)
    irnV2_gavg.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 40, 112, 112, 3)  0         
                             ]                                   
                                                                 
 time_distributed (TimeDistr  (None, 40, 2, 2, 1536)   54336736  
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 40, 1536)         0         
 tributed)                                                       
                                                                 
Total params: 54,336,736
Trainable params: 0
Non-trainable params: 54,336,736
_________________________________________________________________


### ============================ InceptionResNetV2 MAX ============================

In [ ]:
# creating the base model of pre-trained IRNV2 model
from keras.applications import InceptionResNetV2

In [ ]:
tf.debugging.set_log_device_placement(False)
tf.config.set_soft_device_placement(False)
with tf.device('/device:GPU:1'):
    # Load InceptionResNetV2 model
    irnV2 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

    # Freeze ResNet50 layers
    for layer in irnV2.layers:
        layer.trainable = False

    # Extract features using ResNet50
    inputs = Input(shape=INPUT_SHAPE)
    l1 = TimeDistributed(irnV2)(inputs)
    l2 = TimeDistributed(MaxPooling2D())(l1)
    outputs = TimeDistributed(Flatten())(l2)

    # Create model
    irnV2_max = Model(inputs, outputs)
    irnV2_max.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 40, 112, 112, 3)  0         
                             ]                                   
                                                                 
 time_distributed_9 (TimeDis  (None, 40, 2, 2, 1536)   54336736  
 tributed)                                                       
                                                                 
 time_distributed_10 (TimeDi  (None, 40, 1, 1, 1536)   0         
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, 40, 1536)         0         
 stributed)                                                      
                                                                 
Total params: 54,336,736
Trainable params: 0
Non-trainable 

### ============================ MobileNetV2 GAVG ============================

In [ ]:
# creating the base model of pre-trained EffB7 model
from keras.applications import MobileNetV2

In [ ]:
tf.debugging.set_log_device_placement(False)
tf.config.set_soft_device_placement(False)
with tf.device('/device:GPU:1'):
    # Load InceptionResNetV2 model
    mobV2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

    # Freeze ResNet50 layers
    for layer in mobV2.layers:
        layer.trainable = False

    # Extract features using ResNet50
    inputs = Input(shape=INPUT_SHAPE)
    l1 = TimeDistributed(mobV2)(inputs)
    l2 = TimeDistributed(GlobalAveragePooling2D())(l1)
    #outputs = TimeDistributed(Flatten())(l2)

    # Create model
    mobV2_gavg = Model(inputs, l2)
    mobV2_gavg.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 40, 112, 112, 3)  0         
                             ]                                   
                                                                 
 time_distributed (TimeDistr  (None, 40, 4, 4, 1280)   2257984   
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 40, 1280)         0         
 tributed)                                                       
                                                                 
Total params: 2,257,984
Trainable params: 0
Non-trainable params: 2,257,984
_________________________________________________________________


### ============================ DenseNet201 GAVG ============================

In [ ]:
# creating the base model of pre-trained EffB7 model
from keras.applications import DenseNet201

In [ ]:
tf.debugging.set_log_device_placement(False)
tf.config.set_soft_device_placement(False)
with tf.device('/device:GPU:1'):
    # Load DenseNet201 model
    dens201 = DenseNet201(weights='imagenet', include_top=False, input_shape=(112, 112, 3))

    # Freeze ResNet50 layers
    for layer in dens201.layers:
        layer.trainable = False

    # Extract features using ResNet50
    inputs = Input(shape=INPUT_SHAPE)
    l1 = TimeDistributed(dens201)(inputs)
    l2 = TimeDistributed(GlobalAveragePooling2D())(l1)
    #outputs = TimeDistributed(Flatten())(l2)

    # Create model
    dens201_gavg = Model(inputs, l2)
    dens201_gavg.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 40, 112, 112, 3)  0         
                             ]                                   
                                                                 
 time_distributed (TimeDistr  (None, 40, 3, 3, 1920)   18321984  
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 40, 1920)         0         
 tributed)                                                       
                                                                 
Total params: 18,321,984
Trainable params: 0
Non-trainable params: 18,321,984
_________________________________________________________________


## Process

In [ ]:
#class -- ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']

In [ ]:
gc.collect()

0

In [ ]:
category = 'sewing'
GPU = '/device:GPU:1'

In [ ]:
X, y = preprocess_data(category,GPU)
gc.collect()

Loading  sewing
Loading video  1 :  n5E0mGT1g8k_000770_000780
Loading video  3 :  IwU2Gj9tbok_000154_000164
Loading video  5 :  L27AMvA07co_000015_000025
Loading video  7 :  mauZ3bT0CSQ_000053_000063
Loading video  9 :  jK_vvECp2t4_000026_000036
Loading video  11 :  Hh4U-TCx52g_000005_000015
Loading video  13 :  llnEugWEy_Y_000008_000018
Loading video  15 :  JdqnS-vFELM_000003_000013
Loading video  17 :  N6TXdeTKYe0_000900_000910
Loading video  19 :  lpQsuyOYljM_000161_000171
Loading video  21 :  l3p9iVMtwDs_000057_000067
Loading video  23 :  htsM80u3GP4_000000_000010
Loading video  25 :  N60NnV0lhjM_000063_000073
Loading video  27 :  LcMc0Jp7lOY_000021_000031
Loading video  29 :  NAaPjikjmfA_000013_000023
Loading video  31 :  LgDm_6zZelY_000000_000010
Loading video  33 :  N4-_ISQs4u8_000214_000224
Loading video  35 :  hzBjHLViB7A_000017_000027
Loading video  37 :  k5DTFZilFc4_000014_000024
Loading video  39 :  KKcuyBsEYXM_000029_000039
Loading video  41 :  kBuYxiNQrxI_000067_000077
Lo

0

In [ ]:
print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)

Shape of X:  (100, 40, 112, 112, 3)
Shape of y:  (100,)


In [ ]:
sys.getsizeof(X)

602112176

In [ ]:
gc.collect()

0

In [ ]:
np.save(category + '_full_validate', X)
np.save(category + '_full_label_validate', y)

## Extract Feature

### VGG16_AVG

In [ ]:
gc.collect()
model_name = 'vgg16_avg'
X_processed = extract_feature(X, vgg16_avg, 1)
gc.collect()

Start processing...
Processing video  0  -  1


2023-09-06 04:30:01.988996: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,40,224,224,3]
	 [[{{node Placeholder/_0}}]]


Processing video  1  -  2


2023-09-06 04:30:02.396567: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,40,224,224,3]
	 [[{{node Placeholder/_0}}]]


Processing video  2  -  3


...

2023-09-06 07:05:14.544769: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,40,224,224,3]
	 [[{{node Placeholder/_0}}]]


Processing video  596  -  597


2023-09-06 07:05:45.351050: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,40,224,224,3]
	 [[{{node Placeholder/_0}}]]


Processed  597  video


0

### Resnet_GAVG

In [ ]:
model_name = 'resnet_gavg'
X_processed = extract_feature(X, resnet_gavg, 4)
gc.collect()

Start processing...
Processing video  0  -  4
Processing video  4  -  8
Processing video  8  -  12
Processing video  12  -  16
Processing video  16  -  20
Processing video  20  -  24
Processing video  24  -  28
Processing video  28  -  32
Processing video  32  -  36
Processing video  36  -  40
Processing video  40  -  44
Processing video  44  -  48
Processing video  48  -  52
Processing video  52  -  56
Processing video  56  -  60
Processing video  60  -  64
Processing video  64  -  68
Processing video  68  -  72
Processing video  72  -  76
Processing video  76  -  80
Processing video  80  -  84
Processing video  84  -  88
Processing video  88  -  92
Processing video  92  -  96
Processing video  96  -  100
Processed  100  video


0

### VGG16_GAVG

In [ ]:
gc.collect()
model_name = 'vgg16_gavg'
X_processed = extract_feature(X, vgg16_gavg, 2)
gc.collect()

In [ ]:
category = 'yoga'

In [ ]:
X = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Dataset_processed_full/'+category+'_full.npy')
gc.collect()

print('Final shape of X: ', X.shape)
print('Size of X: ', sys.getsizeof(X))

Final shape of X:  (1446, 40, 112, 112, 3)
Size of X:  8706539696


In [ ]:
gc.collect()
model_name = 'vgg16_gavg'
X_processed = extract_feature(X, vgg16_gavg, 4)
gc.collect()

Start processing...
Processing video  0  -  4


2023-09-12 01:31:32.737418: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]
2023-09-12 01:31:32.918900: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


Processing video  4  -  8
Processing video  8  -  12


2023-09-12 01:31:35.435700: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]
2023-09-12 01:31:35.638502: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  12  -  16
Processing video  16  -  20


...

2023-09-12 01:32:53.233412: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  1444  -  1448


2023-09-12 01:32:53.452278: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [2,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processed  1448  video


0

### EFFB7_GAVG

In [ ]:
category = 'watching_tv'

In [ ]:
X = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Dataset_processed_full/'+category+'_full.npy')
gc.collect()

print('Final shape of X: ', X.shape)
print('Size of X: ', sys.getsizeof(X))

Final shape of X:  (1342, 40, 112, 112, 3)
Size of X:  8080343216


In [ ]:
gc.collect()
model_name = 'effB7_gavg'
X_processed = extract_feature(X, effB7_gavg, 4)
gc.collect()

Start processing...
Processing video  0  -  4


2023-09-07 01:18:51.148331: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  4  -  8


2023-09-07 01:18:51.653823: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  8  -  12


...

2023-09-07 01:21:51.916402: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  1340  -  1344


2023-09-07 01:21:52.472748: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [2,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processed  1344  video


0

### IRNV2_GAVG

In [ ]:
category = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']
#category = 'dining'
model_name = 'irnV2_gavg'

In [ ]:
for cat in category:
    X = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_processed_full/'+cat+'_full_validate.npy')
    y = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_processed_full/'+cat+'_full_label_validate.npy')
    gc.collect()

    print('Final shape of X: ', X.shape)
    print('Final shape of y: ', y.shape)

    gc.collect()
    X_processed = extract_feature(X, irnV2_gavg, 4)
    gc.collect()

    print('Final shape of X: ', X_processed.shape)
    print('Size of X: ', sys.getsizeof(X_processed))

    np.save('/home/ycaun/Desktop/jeheng/Numpy/'+ model_name + '_' + cat + '_processed', X_processed)
    np.save('/home/ycaun/Desktop/jeheng/Numpy/'+ model_name + '_' + cat + '_label', y)
    print('Processed data saved to disk')

    del X_processed, X
    gc.collect()

Final shape of X:  (98, 40, 112, 112, 3)
Final shape of y:  (98,)
Start processing...
Processing video  0  -  4


2023-09-08 01:19:59.650199: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  4  -  8


2023-09-08 01:19:59.917677: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  8  -  12


...

Processing video  92  -  96


2023-09-08 01:20:47.191299: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  96  -  100
Processed  100  video


2023-09-08 01:20:47.430982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [2,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Final shape of X:  (98, 40, 1536)
Size of X:  24084624
Processed data saved to disk


In [ ]:
gc.collect()
model_name = 'irnV2_gavg'
X_processed = extract_feature(X, irnV2_gavg, 4)
gc.collect()

Start processing...
Processing video  0  -  4


2023-09-07 01:58:09.653045: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  4  -  8


2023-09-07 01:58:10.004308: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  8  -  12


...

2023-09-07 02:00:23.561118: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  1340  -  1344


2023-09-07 02:00:23.943717: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [2,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processed  1344  video


0

### IRNV2_MAX

In [ ]:
category = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']
model_name = 'irnV2_max'

In [ ]:
for cat in category:
    X = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Dataset_processed_full/'+cat+'_full.npy')
    y = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Dataset_processed_full/'+cat+'_full_label.npy')
    gc.collect()

    print('Loaded ' + cat)
    print('Final shape of X: ', X.shape)
    print('Final shape of y: ', y.shape)

    gc.collect()
    X_processed = extract_feature(X, irnV2_max, 4)
    gc.collect()

    print('Final shape of X: ', X_processed.shape)
    print('Size of X: ', sys.getsizeof(X_processed))

    np.save('/home/ycaun/Desktop/jeheng/Processed dataset/Train_data_processed/'+ model_name + '/' + model_name + '_' + cat + '_processed', X_processed)
    np.save('/home/ycaun/Desktop/jeheng/Processed dataset/Train_data_processed/'+ model_name + '/' + model_name + '_' + cat + '_label', y)
    print('Processed data saved to disk')

    del X_processed, X
    gc.collect()

Loaded dining
Final shape of X:  (1500, 40, 112, 112, 3)
Final shape of y:  (1500,)
Start processing...
Processing video  0  -  4


2023-09-08 03:25:56.500667: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]
2023-09-08 03:25:59.197159: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-09-08 03:26:00.391078: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_1_bfc) ran out of memory trying to allocate 7.60GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-09-08 03:26:00.548770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must 

Processing video  4  -  8
Processing video  8  -  12


2023-09-08 03:26:00.792107: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  12  -  16


...

Processing video  1336  -  1340


2023-09-08 03:39:56.620187: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  1340  -  1344


2023-09-08 03:39:56.932484: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [2,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processed  1344  video
Final shape of X:  (1342, 40, 1536)
Size of X:  329810064
Processed data saved to disk


In [ ]:
del X_processed, X
gc.collect()

795

### MOBV2 GAVG

In [ ]:
category = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']
model_name = 'mobV2_gavg'

In [ ]:
category = ['sewing']
model_name = 'mobV2_gavg'

In [ ]:
for cat in category:
    X = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Dataset_processed_full/'+cat+'_full.npy')
    y = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Dataset_processed_full/'+cat+'_full_label.npy')
    gc.collect()

    print('Loaded ' + cat)
    print('Final shape of X: ', X.shape)
    print('Final shape of y: ', y.shape)

    gc.collect()
    X_processed = extract_feature(X, mobV2_gavg, 4)
    gc.collect()

    print('Final shape of X: ', X_processed.shape)
    print('Size of X: ', sys.getsizeof(X_processed))

    np.save('/home/ycaun/Desktop/jeheng/Processed dataset/Train_data_processed/'+ model_name + '/' + model_name + '_' + cat + '_processed', X_processed)
    np.save('/home/ycaun/Desktop/jeheng/Processed dataset/Train_data_processed/'+ model_name + '/' + model_name + '_' + cat + '_label', y)
    print('Processed data saved to disk')

    del X_processed, X, y
    gc.collect()

Loaded sewing
Final shape of X:  (1440, 40, 112, 112, 3)
Final shape of y:  (1440,)
Start processing...
Processing video  0  -  4


2023-09-09 03:36:55.801779: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]
2023-09-09 03:36:56.479780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


Processing video  4  -  8
Processing video  8  -  12


2023-09-09 03:36:57.157631: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]
2023-09-09 03:36:57.331802: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  12  -  16
Processing video  16  -  20


...

Processing video  1424  -  1428
Processing video  1428  -  1432


2023-09-09 03:38:04.391008: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]
2023-09-09 03:38:04.599438: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  1432  -  1436
Processing video  1436  -  1440


2023-09-09 03:38:04.804091: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]
2023-09-09 03:38:05.007342: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processed  1440  video
Final shape of X:  (1440, 40, 1280)
Size of X:  294912144
Processed data saved to disk


### DENS201 GAVG

In [ ]:
category = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']
model_name = 'dens201_gavg'

In [ ]:
category = ['sewing']
model_name = 'dens201_gavg'

In [ ]:
for cat in category:
    X = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Dataset_processed_full/'+cat+'_full.npy')
    y = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Dataset_processed_full/'+cat+'_full_label.npy')
    gc.collect()

    print('Loaded ' + cat)
    print('Final shape of X: ', X.shape)
    print('Final shape of y: ', y.shape)

    gc.collect()
    X_processed = extract_feature(X, dens201_gavg, 4)
    gc.collect()

    print('Final shape of X: ', X_processed.shape)
    print('Size of X: ', sys.getsizeof(X_processed))

    np.save('/home/ycaun/Desktop/jeheng/Processed dataset/Train_data_processed/'+ model_name + '/' + model_name + '_' + cat + '_processed', X_processed)
    np.save('/home/ycaun/Desktop/jeheng/Processed dataset/Train_data_processed/'+ model_name + '/' + model_name + '_' + cat + '_label', y)
    print('Processed data saved to disk')

    del X_processed, X, y
    gc.collect()

Loaded sewing
Final shape of X:  (1440, 40, 112, 112, 3)
Final shape of y:  (1440,)
Start processing...
Processing video  0  -  4


2023-09-09 04:26:23.887216: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]
2023-09-09 04:26:26.500637: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


Processing video  4  -  8


2023-09-09 04:26:29.087540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  8  -  12


...

Processing video  1432  -  1436


2023-09-09 04:28:13.992758: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  1436  -  1440


2023-09-09 04:28:14.311295: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processed  1440  video
Final shape of X:  (1440, 40, 1920)
Size of X:  442368144
Processed data saved to disk


In [ ]:
y = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Dataset_processed_full/'+category+'_full_label.npy')

In [ ]:
print('Final shape of X: ', X_processed.shape)
print('Size of X: ', sys.getsizeof(X_processed))

Final shape of X:  (1446, 40, 512)
Size of X:  118456464


In [ ]:
np.save('/home/ycaun/Desktop/jeheng/Processed dataset/Train_data_processed/'+ model_name + '/' + model_name + '_' + category + '_processed', X_processed)
np.save('/home/ycaun/Desktop/jeheng/Processed dataset/Train_data_processed/'+ model_name + '/' + model_name + '_' + category + '_label', y)

print('Processed data saved to disk')

Processed data saved to disk


In [ ]:
X_reload = np.load('F:/Kinetic_700/'+ model_name + '/' + category + '_processed.npy')
y_reload = np.load('F:/Kinetic_700/'+ model_name + '/' + category + '_label.npy')

print(X_reload.shape)
print(y_reload.shape)

(98, 40, 2048)
(98,)


### Free Ram

In [ ]:
del X_processed, y
#, y, X_reload, y_reload
gc.collect()

2770

## Split Train Validation

In [ ]:
gc.collect()

0

In [ ]:
def train_test_loader(classname, X_train, X_test, y_train, y_test, model_name, main_dir):

    if len(X_train) != 0 and len(y_train) != 0:
        x_processed = np.load(main_dir + model_name + '/' + model_name + '_' + classname + '_processed.npy')
        split = round(len(x_processed)*0.8)
        X_train = np.concatenate((X_train, x_processed[0:split, :, :]))
        X_test = np.concatenate((X_test, x_processed[split:len(x_processed), :, :]))

        y_processed = np.load(main_dir + model_name + '/' + model_name + '_' + classname + '_label.npy')
        y_train = np.concatenate((y_train, y_processed[0:split]))
        y_test = np.concatenate((y_test, y_processed[split:len(y_processed)]))

        return X_train, X_test, y_train, y_test, x_processed.shape, y_processed.shape, split
    else:
        x_processed = np.load(main_dir + model_name + '/' + model_name + '_' + classname + '_processed.npy')
        split = round(len(x_processed)*0.8)
        X_train = x_processed[0:split, :, :]
        X_test = x_processed[split:len(x_processed), :, :]

        y_processed = np.load(main_dir + model_name + '/' + model_name + '_' + classname + '_label.npy')
        y_train = y_processed[0:split]
        y_test = y_processed[split:len(y_processed)]


        return X_train, X_test, y_train, y_test, x_processed.shape, y_processed.shape, split

In [ ]:
X_train = []
X_test = []
y_train = []
y_test = []
model_name = 'vgg16_gavg'
categories = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv', 'yoga']
main_dir = '/home/ycaun/Desktop/jeheng/Processed dataset/Train_data_processed/'

for classname in categories:
    X_train, X_test, y_train, y_test, current_data_shape, current_label_shape, split = train_test_loader(classname, X_train, X_test, y_train, y_test, model_name, main_dir)

    print("Shape of ", classname, " data set: ", current_data_shape)
    print("Shape of ", classname, " label set: ", current_label_shape)
    print("Shape of X_train: ", X_train.shape)
    print("Shape of y_train: ", y_train.shape)
    print("Shape of X_test: ", X_test.shape)
    print("Shape of y_test: ", y_test.shape, '\n')


Shape of  dining  data set:  (1500, 40, 512)
Shape of  dining  label set:  (1500,)
Shape of X_train:  (1200, 40, 512)
Shape of y_train:  (1200,)
Shape of X_test:  (300, 40, 512)
Shape of y_test:  (300,) 

Shape of  drawing  data set:  (1476, 40, 512)
Shape of  drawing  label set:  (1476,)
Shape of X_train:  (2381, 40, 512)
Shape of y_train:  (2381,)
Shape of X_test:  (595, 40, 512)
Shape of y_test:  (595,) 

Shape of  mopping_floor  data set:  (1688, 40, 512)
Shape of  mopping_floor  label set:  (1688,)
Shape of X_train:  (3731, 40, 512)
Shape of y_train:  (3731,)
Shape of X_test:  (933, 40, 512)
Shape of y_test:  (933,) 

Shape of  reading_book  data set:  (1872, 40, 512)
Shape of  reading_book  label set:  (1872,)
Shape of X_train:  (5229, 40, 512)
Shape of y_train:  (5229,)
Shape of X_test:  (1307, 40, 512)
Shape of y_test:  (1307,) 

Shape of  running_on_treadmill  data set:  (1630, 40, 512)
Shape of  running_on_treadmill  label set:  (1630,)
Shape of X_train:  (6533, 40, 512)
Shap

In [ ]:
y_train[y_train == 'dining'] = 0
y_train[y_train == 'drawing'] = 1
y_train[y_train == 'mopping_floor'] = 2
y_train[y_train == 'reading_book'] = 3
y_train[y_train == 'running_on_treadmill'] = 4
y_train[y_train == 'sleeping'] = 5
y_train[y_train == 'watching_tv'] = 6
y_train[y_train == 'yoga'] = 7

y_test[y_test == 'dining'] = 0
y_test[y_test == 'drawing'] = 1
y_test[y_test == 'mopping_floor'] = 2
y_test[y_test == 'reading_book'] = 3
y_test[y_test == 'running_on_treadmill'] = 4
y_test[y_test == 'sleeping'] = 5
y_test[y_test == 'watching_tv'] = 6
y_test[y_test == 'yoga'] = 7

In [ ]:
y_train = to_categorical(y_train, num_classes=8)
y_test = to_categorical(y_test, num_classes=8)

print("Shape of y_train: ", y_train.shape)
print("Shape of y_test: ", y_test.shape)

Shape of y_train:  (9926, 8)
Shape of y_test:  (2480, 8)


In [ ]:
save_dir = main_dir + model_name + '/Split/'
print(save_dir)

/home/ycaun/Desktop/jeheng/Processed dataset/Train_data_processed/vgg16_gavg/Split/


In [ ]:
np.save(save_dir + 'X_train', X_train)
np.save(save_dir + 'y_train', y_train)
np.save(save_dir + 'X_test', X_test)
np.save(save_dir + 'y_test', y_test)

In [ ]:
del X_train, X_test, y_train, y_test
#, y, X_reload, y_reload
gc.collect()

176

### To load manually

In [ ]:
y = []

for classname in CATEGORIES:
    y, shape = load_processed_label(classname, y)

    print("Shape of ", classname, " set: ", shape)
    print("Shape of y: ", y.shape)

In [ ]:
y[y == 'dining'] = 0
y[y == 'drawing'] = 1
y[y == 'mopping_floor'] = 2
y[y == 'reading_book'] = 3
y[y == 'running_on_treadmill'] = 4
y[y == 'sleeping'] = 5
y[y == 'watching_tv'] = 6

In [ ]:
y_encoded = to_categorical(y, num_classes=7)
print("Shape of y_encoded: ", y_encoded.shape)

In [ ]:
X = []

for classname in CATEGORIES:
    X, shape = load_processed_data(classname, X)

    print("Shape of X: ", X.shape)
    print("Shape of ", classname , ': ', shape)


In [ ]:
print("Shape of X: ", X.shape)
print("Shape of y: ", y_encoded.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)

In [ ]:
np.save('X_train', X_train)
np.save('y_train', y_train)
np.save('X_test', X_test)
np.save('y_test', y_test)

### Load validation set

In [ ]:
CATEGORIES = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv','yoga']

In [ ]:
def load_processed_data(category, X, model):
    if len(X) != 0:
        x_processed = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_data_processed/'+ model_name + '/' + model_name + '_' + category + '_processed.npy')
        X = np.concatenate((X, x_processed))
        return X, x_processed.shape
    else:
        X = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_data_processed/'+ model_name + '/' + model_name + '_' + category + '_processed.npy')
        return X, X.shape

In [ ]:
def load_processed_label(category, y, model):
    if len(y) != 0:
        y_processed = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_data_processed/'+ model_name + '/' + model_name + '_' + category + '_label.npy')
        y = np.concatenate((y, y_processed))
        return y, y_processed.shape
    else:
        y = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_data_processed/'+ model_name + '/' + model_name + '_' + category + '_label.npy')
        return y, y.shape

In [ ]:
def one_hot(y):
    y[y == 'dining'] = 0
    y[y == 'drawing'] = 1
    y[y == 'mopping_floor'] = 2
    y[y == 'reading_book'] = 3
    y[y == 'running_on_treadmill'] = 4
    y[y == 'sleeping'] = 5
    y[y == 'watching_tv'] = 6
    y[y == 'yoga'] = 7

    y_encode = to_categorical(y, num_classes=8)

    return y_encode

In [ ]:
category = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']
model_name = 'vgg16_gavg'

In [ ]:
for cat in category:
    X = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_processed_full/'+cat+'_full_validate.npy')
    y = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_processed_full/'+cat+'_full_label_validate.npy')
    gc.collect()

    print('Loaded ' + cat)
    print('Final shape of X: ', X.shape)
    print('Final shape of y: ', y.shape)

    gc.collect()
    X_processed = extract_feature(X, vgg16_gavg, 4)
    gc.collect()

    print('Final shape of X: ', X_processed.shape)
    print('Size of X: ', sys.getsizeof(X_processed))

    np.save('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_data_processed/'+ model_name + '/' + model_name + '_' + cat + '_processed', X_processed)
    np.save('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_data_processed/'+ model_name + '/' + model_name + '_' + cat + '_label', y)
    print('Processed data saved to disk')

    del X_processed, X, y
    gc.collect()

Loaded dining
Final shape of X:  (98, 40, 112, 112, 3)
Final shape of y:  (98,)
Start processing...
Processing video  0  -  4


2023-09-12 01:41:34.540434: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  4  -  8
Processing video  8  -  12


2023-09-12 01:41:34.756576: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]
2023-09-12 01:41:34.971187: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  12  -  16


...

Processing video  92  -  96


2023-09-12 01:42:15.028772: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [4,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Processing video  96  -  100
Processed  100  video


2023-09-12 01:42:15.238135: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [2,40,112,112,3]
	 [[{{node Placeholder/_0}}]]


Final shape of X:  (98, 40, 512)
Size of X:  8028304
Processed data saved to disk


In [ ]:
X_eval = []
y_eval = []

for classname in CATEGORIES:
    X_eval, X_current_shape = load_processed_data(classname,X_eval, 'vgg16_gavg')
    y_eval, y_current_shape = load_processed_label(classname,y_eval, 'vgg16_gavg')
    y_encode = one_hot(y_eval)

    print("Shape of", classname, " data set: ", X_current_shape)
    print("Shape of", classname, " label set: ", y_current_shape)
    print("Shape of X_eval: ", X_eval.shape)
    print("Shape of y_encode: ", y_encode.shape, '\n')

Shape of dining  data set:  (98, 40, 512)
Shape of dining  label set:  (98,)
Shape of X_eval:  (98, 40, 512)
Shape of y_encode:  (98, 8) 

Shape of drawing  data set:  (92, 40, 512)
Shape of drawing  label set:  (92,)
Shape of X_eval:  (190, 40, 512)
Shape of y_encode:  (190, 8) 

Shape of mopping_floor  data set:  (100, 40, 512)
Shape of mopping_floor  label set:  (100,)
Shape of X_eval:  (290, 40, 512)
Shape of y_encode:  (290, 8) 

Shape of reading_book  data set:  (98, 40, 512)
Shape of reading_book  label set:  (98,)
Shape of X_eval:  (388, 40, 512)
Shape of y_encode:  (388, 8) 

Shape of running_on_treadmill  data set:  (96, 40, 512)
Shape of running_on_treadmill  label set:  (96,)
Shape of X_eval:  (484, 40, 512)
Shape of y_encode:  (484, 8) 

Shape of sleeping  data set:  (96, 40, 512)
Shape of sleeping  label set:  (96,)
Shape of X_eval:  (580, 40, 512)
Shape of y_encode:  (580, 8) 

Shape of watching_tv  data set:  (98, 40, 512)
Shape of watching_tv  label set:  (98,)
Shape o

In [ ]:
np.save(model_name+'_X_eval', X_eval)
np.save(model_name+'_y_eval', y_encode)

In [ ]:
y = np.load('/home/ycaun/Desktop/jeheng/Processed dataset/Validate_data_processed/irnV2_max/irnV2_max_y_eval.npy')
y_encode = one_hot(y)
print("Shape of y_encode: ", y_encode.shape, '\n')
np.save('irnV2_max_y_eval', y_encode)

Shape of y_encode:  (678, 7) 



In [ ]:
y_eval = []

for classname in CATEGORIES:
    y_eval, y_current_shape = load_processed_label(classname,y_eval, 'mobV2_gavg')
    y_encode = one_hot(y_eval)

    print("Shape of", classname, " label set: ", y_current_shape)
    print("Shape of y_encode: ", y_encode.shape, '\n')

Shape of dining  label set:  (98,)
Shape of y_encode:  (98, 9) 

Shape of drawing  label set:  (92,)
Shape of y_encode:  (190, 9) 

Shape of mopping_floor  label set:  (100,)
Shape of y_encode:  (290, 9) 

Shape of reading_book  label set:  (98,)
Shape of y_encode:  (388, 9) 

Shape of running_on_treadmill  label set:  (96,)
Shape of y_encode:  (484, 9) 

Shape of sleeping  label set:  (96,)
Shape of y_encode:  (580, 9) 

Shape of watching_tv  label set:  (98,)
Shape of y_encode:  (678, 9) 

Shape of yoga  label set:  (100,)
Shape of y_encode:  (778, 9) 

Shape of sewing  label set:  (100,)
Shape of y_encode:  (878, 9) 



In [ ]:
np.save(model_name+'_y_eval', y_encode)